Rename a bunch of files using Git and renamer?



In [ ]:
var renamer = require('renamer');

function renameUsingGit(gitRoot, match, find, replace) {
    var files = renamer.expand(path.join(gitRoot, match));
    var results = renamer.replace({
        files: files.filesAndDirs,
        find: find,
        replace: replace
    });
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git '
            + '--work-tree=' + JSON.stringify(gitRoot)
            + ' --git-dir=' + JSON.stringify(path.join(gitRoot, '.git'))
            + ' mv ' + JSON.stringify(r.before)
            + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
renameUsingGit;


In [ ]:
// test it
// rename all scss files to less
//renameUsingGit(
//    '/Users/briancullinan/Documents/studysauce4',
//    'src/**/*.css',
//    '.css',
//    '.scss');


json git tree?


In [ ]:
var execSync = require('child_process').execSync;

function gitTree(project) {
    const dir = process.cwd();
    process.chdir(project); 
    var branches = execSync('git branch --list --all ').toString()
        .trim().split(/\s*\n\s*\**\s*/ig)
        .map(b => b.replace('remotes/', '').trim());

    // reverse git so we can add merges to the "graph" after the branch tree is constructed
    var branch = execSync('git log --pretty=format:"%ct %d %h %p" '
        + '--reverse --branches --tags ').toString().split(/\s*\n\s*/ig);
    var all = []; //execSync('git log --pretty=format:"%h %p" --reverse --all').toString().split(/\s*\n\s*/ig);

    var nodes = [];
    var edges = [];
    branch.forEach(b => {
        var commits = ((/[0-9]+\s+\(.*?\)\s*(.+)/ig).exec(b)
            || (/[0-9]+\s+(.+)/ig).exec(b) || [b, b.trim()])[1].trim().split(/\s+/ig);
        commits.forEach(r => {
            if (nodes.filter(d => d.name == r).length == 0) {
                nodes[nodes.length] = {
                    name: r,
                    index: nodes.length,
                    branch: ((/[0-9]+\s+\((.*?)\)/ig).exec(b) || [b, ''])[1],
                    time: (/([0-9]+)\s+/ig).exec(b)[1]
                }
            }
        });
        // don't include merges in tree
        if (commits.length == 3) {
            commits = [commits[0], commits[2]];
        }
        if (commits.length == 2) {
            edges = edges.concat(commits.slice(1)
            // check all existing sources to test for merge
                .map((c, i) => {
                    return {source: c, target: commits[i]}; // -1 because of slice(1)
                }))
        }
    });

    process.chdir(dir);
    return formatNodes(nodes, edges.reverse());
}
module.exports = gitTree;
gitTree;



tip git tree?


In [10]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var path = require('path');
var execSync = require('child_process').execSync;
var gulp = require('gulp');
var copy = require('gulp-copy');
var clean = require('gulp-clean');
var {
    formatNodes, gitTree, displayBranches,
    tasksToPromise, listInProject, formatNodes
} = importer.import([
    'd3.ipynb[format d3 tree]',
    'gulp tasks',
    'json git tree',
    'd3.ipynb[display d3 tree]',
    'list files project'
], {gulp});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project_orig = PROFILE_PATH + '/Documents/portal';
var project = PROFILE_PATH + '/Documents/portal3';
process.chdir(PROFILE_PATH + '/Documents');

gulp.task('clear project', function () {
    return gulp.src(project, {read: false})
        .pipe(clean({force: true}));
});

gulp.task('copy index', function () {
    return gulp.src(project_orig)
        .pipe(copy(project))
        .dest(project);
});

function getResetCommit(branch, parent) {
    const b = branch.replace('HEAD -> ', '').trim();
    execSync('git checkout ' + b);
    execSync('git reset --mixed ' + parent.replace('HEAD -> ', '').trim());
    execSync('git add -A')
    execSync('git commit -m "auto reset ' + b + '"')
}

var svgs = [];
function resetAllBranches() {
    var children = [];
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    var promises = [];
    for (const n of nodeTree) {
        var current = n;
        var parentCount = 0;
        while (typeof current.parent !== 'undefined'
        && current.parent.branch === current.branch) {
            current = current.parent;
            parentCount++;
        }
        if (parentCount > 0 && typeof current.parent !== 'undefined') {
            promises.push(resolve => {
                try {
                    if (typeof n.children !== 'undefined') {
                        children.push(n);
                    }
                    getResetCommit(n.branch, current.parent.name);
                } finally {
                    var nodeTree = _.sortBy(gitTree(project), a => a.time);
                    svgs.push(displayBranches(nodeTree));
                    resolve();
                }
            });
        }
    }
    return importer.runAllPromises(promises)
        .then(() => children);
}

function gitTipOfTree() {
    svgs = [];
    return (fs.existsSync(project) ? tasksToPromise(['clear project']) : Promise.resolve([]))
        .then(() => Promise.all([
            listInProject(project_orig, '**/*'),
            listInProject(project_orig + '/.git', '**/*')
        ]))
        .then(r => {
            for (const file of [].concat(...r).sort()) {
                const dirname = path.dirname(
                    file.replace(project_orig, project)).split('/');

                for (var i = 2; i <= dirname.length; i++) {
                    if (!fs.existsSync(dirname.slice(0, i).join('/'))) {
                        fs.mkdirSync(dirname.slice(0, i).join('/'));
                    }
                }
                fs.copyFileSync(file, file.replace(project_orig, project));
            }
            process.chdir(project);
        })
        .then(() => {
            var nodeTree = _.sortBy(gitTree(project), a => a.time);
            return displayBranches([nodeTree[0]]);
        })
        .then(r => svgs.push(r))
        //.then(() => resetAllBranches())
        .then(() => svgs)
};
module.exports = gitTipOfTree;


[Function: gitTipOfTree]

In [64]:
var importer = require('../Core');
var gitTipOfTree = importer.import('git.ipynb[tip git tree]');

$$.async();
gitTipOfTree()
    .then(r => $$.html(`<style>
div.svg-wrapper svg {
    max-width:none;
}

div.svg-output {
    min-height:600px;
}
div.svg-wrapper:first-of-type {
    position:static;
}
div.svg-wrapper {
    opacity: .05;
    position:absolute;
}

div.svg-wrapper:nth-last-of-type(3) {
    opacity: .15;
}

div.svg-wrapper:nth-last-of-type(2) {
    opacity: .3;
}

div.svg-wrapper:last-of-type {
    opacity: 1;
}
</style>
<div class="svg-output"><div
` + r.map((s, i) => ' style="left:' + ((r.length - i) * -20 + (i == r.length - 1
        ? 200
        : 0)
        + (i == r.length - 2
            ? 150
            : 0)
        + (i == r.length - 3
            ? 100
            : 0)) + 'px; top:' + (i * 10) + 'px;" class="svg-wrapper">' + s).join('</div><div') + '</div></div>'))
    .catch(e => $$.sendError(e))


4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) fee839d a229417 6a552aa af88ea2 ( swiftpage/Only_auth, Only_auth ) a8c1fd7 2c184e7 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 67b6565 7fa4cf6 6141fc7 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) 580ee75 bcc334f 8038f45 e3cae80 86fcb7e 386b890 ( swiftpage/B-11309_APC_Direct_Purchase, B-11309_APC_Direct_Purchase ) 4f6b93b 4eac003 8a82bc9 ( swiftpage/development ) bb3ab05 ( B-11376_Account_Management_2 ) 498d5a5 cd9039d ( swiftpage/B-11649_Convert_Product_Ids_To_Constants ) 620fcf1 ( Universal ) f47691d 372983c ( HEAD -> B-11650_Culture_From_URL, swiftpage/B-11650_Culture_From_URL ) b4da990 b8ca286 ( swiftpage/B11583_Portal_2_0_Sell_Pro_Direct_through_Portal_Phase_1, B11583_Portal_2_0_Sell_Pro_Direct_through_Portal_Phase_1 )

Display graphs of different commands and scenarios using dry-run?

https://github.com/kpdecker/jsdiff

http://incaseofstairs.com/jsdiff/



In [ ]:
var path = require('path');
var gulp = require('gulp');
var run = require('gulp-run');
var tap = require('gulp-tap');
var importer = require('../Core');
var gulpPromise = importer.import('gulp task to a Promise', {gulp})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;

process.chdir(project)

gulp.task('git watch', function () {
    return run('git diff -w a229417..498d5a5')
        .pipe(tap(file => console.log(file)))
        .pipe(gulp.dest('output'));
});

function getBranchBoundaries(p) {
    project = p;
    return gulpPromise(['git watch'])
}
module.exports = getBranchBoundaries;
getBranchBoundaries;

$$.async();
getBranchBoundaries(project)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



TODO:

Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?

Load relevant projects based on branch.  It could parse the .sln file and automatically patch whenever the checked out index changes.

Plot when a TODO: item is added and when it is removed on Google Calendar, then verify by checking commit -messages, remaining comments/function names with matching keywords (split by camel casing)?, plot dates completed for measurement of self set tasks and short term goals, which activities are harder than others?  better insight in to erroneous developer time estimates?

Interface for searching all TODO: items?  Calendar?

